This model differs from base model in that it uses only the source data to make a prediction, doesnt remove or transform any variables, no feature engineering

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
 
sys.path.append('C:\\Users\\Nikhil\\Data_Science_Projects')
from common_ds_modules import missing_values, data_manipulation, modeling, eda
import os
import pandas as pd
import numpy as np
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import pickle
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier

from sklearn.compose import make_column_transformer 
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import FunctionTransformer
log_transformer = FunctionTransformer(np.log1p)

from sklearn.compose import make_column_selector as selector
from scipy.stats import skew
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import VotingClassifier

In [3]:
MAX_MODELS = 50

In [4]:
def get_columns(df, query=''):
    return [c for c in df.columns if query.lower() in c.lower()]

In [35]:
def get_pipeline_cross_val_score(preprocessor, model, param_grid, x_train, y_train, scoring_function,max_models=None):
    print(f'Max models: {max_models}')
    if model.__name__ == 'Lasso' or model.__name__ == 'Ridge' or model.__name__ == 'ElasticNet':
        print(f'Scaling Pipe: {model.__name__}')
        pipe = make_pipeline(preprocessor, StandardScaler(), model())
    else:
        pipe = make_pipeline(preprocessor, model())
    
    if max_models is not None:
        random_search = RandomizedSearchCV(pipe, param_grid, scoring=scoring_function,
                                   n_iter=max_models, verbose=0)
    else:
        random_search = GridSearchCV(pipe, param_grid, scoring=scoring_function,
                                   n_iter=max_models, verbose=0)

    random_search = random_search.fit(x_train, y_train)
    best_params = { k.split('__')[1]:v for (k,v) in zip(list(random_search.best_params_.keys()), list(random_search.best_params_.values()))} 
    if model.__name__ == 'Lasso' or model.__name__ == 'Ridge' or model.__name__ == 'ElasticNet':
        print(f'Scaling Final Pipe: {model.__name__}')
        final_pipe = final_pipe = make_pipeline(preprocessor, StandardScaler(), model(**best_params))
    else:
        final_pipe = make_pipeline(preprocessor, model(**best_params))
    
    score = cross_val_score(final_pipe, x_train, y_train, cv=5, scoring=scoring_function).mean()
    
    return final_pipe, score, best_params

In [6]:
xgb_param_grid = {
    'xgbclassifier__n_estimators': [110, 120],
    'xgbclassifier__max_depth':[5,6],
    'xgbclassifier__max_leaves':[40, 30, 50],
    'xgbclassifier__reg_alpha': [i/100 for i in range(1,3)],
    'xgbclassifier__reg_lambda': [i/100 for i in range(1,3)],
    'xgbclassifier__colsample_bytree': [i/10 for i in range(2, 5)],
    'xgbclassifier__learning_rate': [i/100 for i in range(3,6)],
    'xgbclassifier__gamma': [i/100 for i in range(1, 3)]
}

lasso_param_grid = {
    'lasso__alpha':[0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

ridge_param_grid = {
    'ridge__alpha':[0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

elastic_net_param_grid = {
    'elasticnet__alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'elasticnet__l1_ratio':[0.01, 0.05, 0.1, 0.5, 1]
}

knn_param_grid = {
    'kneighborsclassifier__weights' : ['uniform'],
    'kneighborsclassifier__algorithm' : ['auto'],
    'kneighborsclassifier__n_neighbors' : [5,10,15, 20, 25, 30],
    'kneighborsclassifier__leaf_size': [10],
    'kneighborsclassifier__p': [1],
}

In [7]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [8]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
label = 'Survived'
y_train = train_df[label]

In [10]:
id_column = 'PassengerId'
pred_col = f'Predicted_{label}'

In [11]:
numerical_variables, categorical_variables = data_manipulation.get_numerical_categorical_variables(train_df,\
                                                                                            test_df, id_column)                                                                                                  

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

In [13]:
def plot_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].describe())
        plt.hist(df[c])
        plt.title(f'Distribution for {c}')
        plt.show()
        
def plot_discrete_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].value_counts(normalize=True))
        plt.hist(df[c])
        plt.title(f'Distribution for {c}')
        plt.show()
        
def plot_cat_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].value_counts(normalize=True))
        plt.bar(df[c].value_counts().index, df[c].value_counts())
        plt.title(f'Distribution for {c}')
        plt.show()

In [14]:
def predict(x_train, y_train, x_test, model, id_col_name, id_col, pred_col, label, filename):
    model = model.fit(x_train, y_train)
    test_pred = model.predict(x_test)
    
    x_test[pred_col] = test_pred
    x_test[id_col_name] = id_col
    
    pred = x_test[[id_col_name, pred_col]]
    columns = dict()
    columns[pred_col] = label
    pred = pred.rename(columns=columns)
    pred.to_csv(filename, index=False)

In [15]:
missing_value_df = missing_values.get_variable_missing_values(train_df).reset_index(drop=True).sort_values('percent_missing', ascending=False)
missing_value_df

,column_name,percent_missing
10,Cabin,77.104377
5,Age,19.865320
11,Embarked,0.224467
0,PassengerId,0.000000
1,Survived,0.000000
2,Pclass,0.000000
3,Name,0.000000
4,Sex,0.000000
6,SibSp,0.000000
7,Parch,0.000000


In [16]:
train_df2 = train_df.copy()
test_df2 = test_df.copy()

In [17]:
id_column

'PassengerId'

In [18]:
def get_preprocessor(train_df, id_column, label):
    numerical_imputer = SimpleImputer(strategy='mean')
    categorical_imputer = SimpleImputer(strategy='most_frequent')

    categorical_variables, discrete_numerical_variables,\
    continuous_numerical_variables = data_manipulation.get_variables(train_df, [], id_column)
    discrete_numerical_variables = list(set(discrete_numerical_variables) - set([label])) 
    ind_continuous_variables = list(set(continuous_numerical_variables) - set([label]))
    numerical_variables = continuous_numerical_variables + discrete_numerical_variables

    categorical_columns_selector = selector(dtype_exclude='number')
    categorical_columns = categorical_columns_selector(train_df)
    categorical_columns = list(set(categorical_columns) - set([label]))
    #categorical_columns = ['Embarked', 'Ticket', 'Sex', 'Cabin']
    ohe2 = OneHotEncoder(handle_unknown='ignore', sparse=False)
    categorical_pipeline2 = make_pipeline(categorical_imputer, ohe2)
    
    print(f'Continuous Variables: {ind_continuous_variables}')
    print(f'Discrete Numerical Variables: {discrete_numerical_variables}')
    print(f'Categorical Variables: {categorical_columns}')
    
    preprocessor = make_column_transformer((categorical_pipeline2, categorical_columns),
                                           (numerical_imputer, ind_continuous_variables),
                                           (categorical_imputer, discrete_numerical_variables),
                                        )
    return preprocessor

In [19]:
def train_stacking_classifier(train_df, y_train, estimators, final_estimator, scoring_function):
    stacking_classifier = StackingClassifier(
            estimators=estimators,
            final_estimator=final_estimator,
        )
    
    score = cross_val_score(stacking_classifier, train_df, y_train, cv=5, scoring=scoring_function).mean()
    print(f'Score: {score}')
    return stacking_classifier, score


In [20]:
def train_classifiers(preprocessor, train_df, y_train, dt_param_grid, rf_param_grid, knn_param_grid, xgb_param_grid,\
                    gbr_param_grid):
    
    dt_final_pipe, dt_score, dt_best_params = get_pipeline_cross_val_score(preprocessor, DecisionTreeClassifier,\
                                            dt_param_grid, train_df, y_train, 'accuracy', 50)
    print(f'DT Score: {dt_score}, dt best params: {dt_best_params}')
    print()
    print()
    
    rf_final_pipe, rf_score, rf_best_params = get_pipeline_cross_val_score(preprocessor, RandomForestClassifier,\
                                                            rf_param_grid, train_df, y_train, 'accuracy', 50)
    print(f'RF Score: {rf_score}, RF best params: {rf_best_params}')
    print()
    print()
    
    knn_final_pipe, knn_score, knn_best_params = get_pipeline_cross_val_score(preprocessor, KNeighborsClassifier,\
                                                knn_param_grid, train_df, y_train, 'accuracy', 50)
    print(f'KNN Score: {knn_score}, KNN best params: {knn_best_params}')
    print()
    print()
    
    xgb_final_pipe, xgb_score, xgb_best_params = get_pipeline_cross_val_score(preprocessor, XGBClassifier, xgb_param_grid,\
                                                train_df, y_train, 'accuracy', 50)
    print(f'XGB Score: {xgb_score}, XGB best params: {xgb_best_params}')
    print()
    print()
    
    gbr_final_pipe, gbr_score, gbr_best_params = get_pipeline_cross_val_score(preprocessor, GradientBoostingClassifier,\
                                                gbr_param_grid, train_df, y_train, 'accuracy', 50)
    print(f'GBR Score: {gbr_score}, GBR best params: {gbr_best_params}')
    print()
    print()
    
    lr_final_pipe = make_pipeline(preprocessor, LogisticRegression())
    lr_score = cross_val_score(lr_final_pipe, train_df, y_train, cv=5, scoring='accuracy').mean()
    print(f'Logistic Regression Score: {lr_score}')
    print()
    print()
    
    return {'DecisionTree': dt_final_pipe, 'RandomForest': rf_final_pipe, 'KNN': knn_final_pipe, 'XGB': xgb_final_pipe,\
            'GBR': gbr_final_pipe, 'LogisticRegression': lr_final_pipe }

Code to train stack classifiers

In [21]:
%%time
dt_param_grid = {
    'decisiontreeclassifier__max_depth':[2,3],
    'decisiontreeclassifier__min_samples_split':[10, 20, 30],
    'decisiontreeclassifier__min_samples_leaf':[10, 20, 30],
    'decisiontreeclassifier__max_features' : ['sqrt'],
    'decisiontreeclassifier__min_impurity_decrease': [i/100 for i in range(1, 3)],
    'decisiontreeclassifier__ccp_alpha': [i/10 for i in range(1, 3)]
}

rf_param_grid = {
    'randomforestclassifier__n_estimators':[100, 500, 1000],
    'randomforestclassifier__min_samples_split':[10, 20, 30, 40, 50],
    'randomforestclassifier__max_depth': [2,3,4,5],
    'randomforestclassifier__min_samples_leaf': [5, 10, 15, 20, 25, 30],
    'randomforestclassifier__min_impurity_decrease': [0.002, 0.004, 0.006, 0.008]
}

knn_param_grid = {
    'kneighborsclassifier__weights' : ['uniform'],
    'kneighborsclassifier__algorithm' : ['auto'],
    'kneighborsclassifier__n_neighbors' : [5,10,15, 20, 25, 30],
    'kneighborsclassifier__leaf_size': [10],
    'kneighborsclassifier__p': [1],
}

xgb_param_grid = {
    'xgbclassifier__n_estimators': [110, 120],
    'xgbclassifier__max_depth':[5,6],
    'xgbclassifier__max_leaves':[40, 30, 50],
    'xgbclassifier__reg_alpha': [i/100 for i in range(1,3)],
    'xgbclassifier__reg_lambda': [i/100 for i in range(1,3)],
    'xgbclassifier__colsample_bytree': [i/10 for i in range(2, 5)],
    'xgbclassifier__learning_rate': [i/100 for i in range(3,6)],
    'xgbclassifier__gamma': [i/100 for i in range(1, 3)]
}

gbr_param_grid = {
    'gradientboostingclassifier__n_estimators': [1000, 1200],
    'gradientboostingclassifier__max_depth':[4,5,6],
    'gradientboostingclassifier__max_leaf_nodes':[40, 30, 50],
    'gradientboostingclassifier__min_samples_split':[40, 30, 50],
    'gradientboostingclassifier__subsample': [i/10 for i in range(2, 5)],
    'gradientboostingclassifier__learning_rate': [i/100 for i in range(3,6)],
    'gradientboostingclassifier__max_features': [i/100 for i in range(1, 3)]
}

CPU times: total: 0 ns
Wall time: 0 ns


#### Base KNN Classifier

In [22]:
#predict(train_df3, y_train, test_df3, knn_final_pipe, id_column, pred_col, label, 'knn_clf_v2.csv')

In [23]:
def get_headers(worksheet, row_index):
    num_rows = worksheet.max_row
    num_cols = worksheet.max_column
    headers = []
    for i in range(row_index, row_index + 1):
        for j in range(1, num_cols + 1):
            cell = worksheet.cell(row=i, column=j)
            if cell.value:
                #print(f'Cell at ({i}, {j}): {cell.value}')
                headers.append(cell.value)
    return headers

change it to use pandas

In [24]:
def save_results(workbook, worksheet_name, data, row_index, workbook_title):
    worksheet = workbook[worksheet_name]
    current_row = worksheet.max_row + 1
    headers = get_headers(worksheet, row_index)
    num_cols = len(headers)
    print(f'Num columns: {num_cols}')
    print(f'current row: {current_row}')
    cell = worksheet.cell(row=current_row - 1, column=1)
    prev_version = 0
    if cell.value != 'Version':
        current_version = cell.value + 1
        current_cell = worksheet.cell(row=current_row, column=1)
        current_cell.value = current_version
    else:
        current_cell = worksheet.cell(row=current_row, column=1)
        current_cell.value = 1
        
    for j in range(2, num_cols + 1):
        cell = worksheet.cell(row=current_row, column=j)
        cell.value = str(list(data.values())[j - 3])
        print(f'Current Row = {current_row}, current column: {j}, cell_value: {cell.value}')
    workbook.save(workbook_title)

In [44]:
def train_classifier(preprocessor, model, hyper_parameter_set, train_df, y_train, scoring_function, max_models,\
                         workbook, worksheet_name, workbook_name):
    
    final_pipe, train_score, best_params = get_pipeline_cross_val_score(preprocessor, model,\
                                            hyper_parameter_set, train_df, y_train, scoring_function, max_models)
    
    data = dict()
    feature_list = list(train_df.columns)
    feature_str = ',\n '.join(feature_list)
    
    print(f'train_score: {train_score}')
    
    #workbook_name = 'Titanic Survival Model.xlsx'
    sheet = pd.read_excel(workbook_name, sheet_name=worksheet_name).rename(columns=lambda x: x.strip())
    
    if len(sheet) > 0:
        old_version = sheet.loc[len(sheet) - 1, 'Version']
        new_version = old_version + 1
    else:
        new_version = 1
        
    public_score = 0
    sheet.loc[len(sheet)] = [new_version, feature_str, hyper_parameter_set, best_params, train_score, public_score] # row

    with pd.ExcelWriter(workbook_name,
                        mode='a', if_sheet_exists='replace') as writer:  
        sheet.to_excel(writer, sheet_name=worksheet_name, index=False)
    #row_index = 1
    #save_results(workbook, worksheet_name, data, row_index, workbook_name)
    
    return final_pipe

In [26]:
from openpyxl import load_workbook

In [27]:
workbook = load_workbook('Titanic Survival Model.xlsx')

In [28]:
"""worksheet = workbook['KNN']
current_row = worksheet.max_row + 1
row_index = 1
headers = get_headers(worksheet, row_index)
num_cols = len(headers)
print(f'Num columns: {num_cols}')

cell = worksheet.cell(row=current_row, column=1)
if cell.value:
    prev_version = cell.value
else:
    prev_version = 1
print(f'cell value: {cell.value}')
#if worksheet.cell(row=current_row - 1, column=1).value"""

"worksheet = workbook['KNN']\ncurrent_row = worksheet.max_row + 1\nrow_index = 1\nheaders = get_headers(worksheet, row_index)\nnum_cols = len(headers)\nprint(f'Num columns: {num_cols}')\n\ncell = worksheet.cell(row=current_row, column=1)\nif cell.value:\n    prev_version = cell.value\nelse:\n    prev_version = 1\nprint(f'cell value: {cell.value}')\n#if worksheet.cell(row=current_row - 1, column=1).value"

In [29]:
def get_columns(obj, query=''):
    for field in dir(obj):
        if field[0:2] != '__' and field[0] != '_':
            if query.lower() in field.lower():
                print(f'Field: {field}')

In [30]:
def get_categorical_columns(train_df, label):
    categorical_columns_selector = selector(dtype_exclude='number')
    categorical_columns = categorical_columns_selector(train_df)
    categorical_columns = list(set(categorical_columns) - set([label]))
    
    return categorical_columns

In [31]:
def get_numerical_variables(train_df, id_column, label):
    categorical_variables, discrete_numerical_variables,\
    continuous_numerical_variables = data_manipulation.get_variables(train_df, [], id_column)
    discrete_numerical_variables = list(set(discrete_numerical_variables) - set([label])) 
    ind_continuous_variables = list(set(continuous_numerical_variables) - set([label]))
    
    return ind_continuous_variables, discrete_numerical_variables

In [32]:
ind_continuous_variables, discrete_numerical_variables = get_numerical_variables(train_df2, id_column, label)
categorical_columns = get_categorical_columns(train_df2, label)

In [33]:
variables = categorical_columns + ind_continuous_variables + discrete_numerical_variables
train_df3 = train_df2[variables]
test_df3 = test_df2[variables]
test_df3[id_column] = test_df2[id_column]

In [34]:
preprocessor = get_preprocessor(train_df3, id_column, label)

Continuous Variables: ['Age', 'Fare']
Discrete Numerical Variables: ['Pclass', 'Parch', 'SibSp']
Categorical Variables: ['Cabin', 'Sex', 'Embarked', 'Name', 'Ticket']


In [45]:
%%time
dt_final_pipe = train_classifier(preprocessor, DecisionTreeClassifier, dt_param_grid, train_df3, y_train, 'accuracy', 50,\
                         workbook, 'DecisionTree', 'Titanic Survival Model.xlsx')

Max models: 50
train_score: 0.6161634548992531
CPU times: total: 5.16 s
Wall time: 6.45 s


In [41]:
workbook_name = 'Titanic Survival Model.xlsx'
sheet = pd.read_excel(workbook_name, sheet_name='DecisionTree').rename(columns=lambda x: x.strip())
sheet

,Version,Features,Hyperparameter Set,Best Values,Accuracy,Public Accuracy


In [49]:
len(sheet)

1

In [66]:
sheet.columns

Index(['Version', 'Features', 'Hyperparameter Set', 'Best Values', 'Accuracy',
       'Public Accuracy'],
      dtype='object')

In [69]:
sheet

,Version,Features,Hyperparameter Set,Best Values,Accuracy,Public Accuracy
0,1,tepig,a,0.9,0.8,0.7


1

In [72]:
feature_list = list(train_df3.columns)
feature_str = ',\n '.join(feature_list)
feature_str

'Cabin,\n Embarked,\n Ticket,\n Sex,\n Name,\n Age,\n Fare,\n Pclass,\n Parch,\n SibSp'

In [76]:
workbook_name = 'Titanic Survival Model.xlsx'
sheet = pd.read_excel(workbook_name, sheet_name='DecisionTree').rename(columns=lambda x: x.strip())

old_version = sheet.loc[len(sheet) - 1, 'Version']
sheet.loc[len(sheet)] = [old_version + 1, feature_str, 'a', 0.91, 0.81, 0] # row

with pd.ExcelWriter('Titanic Survival Model.xlsx',
                    mode='a', if_sheet_exists='replace') as writer:  
    sheet.to_excel(writer, sheet_name='DecisionTree', index=False)

#how to append rows to excel spreadsheet
#sheet.to_excel(workbook_name, sheet_name='DecisionTree')

In [37]:
type(dt_sheet)

pandas.core.frame.DataFrame

In [77]:
sheet

,Version,Features,Hyperparameter Set,Best Values,Accuracy,Public Accuracy
0,1,tepig,a,0.90,0.80,0.70
1,2,"Cabin,\n Embarked,\n Ticket,\n Sex,\n Name,\n ...",a,0.91,0.81,0.71
2,3,"Cabin,\n Embarked,\n Ticket,\n Sex,\n Name,\n ...",a,0.91,0.81,0.00


In [40]:
type(writer)

pandas.io.excel._openpyxl.OpenpyxlWriter

In [53]:
dt_sheet

,Version,Features,Hyperparameter Set,Best Values,Accuracy,Public Accuracy
0,1,tepig,a,0.9,0.8,0.7


In [38]:
pd.read_excel('Titanic Survival Model.xlsx', sheet_name='DecisionTree')

,Version,Features,Hyperparameter Set,Best Values,Accuracy,Public Accuracy
0,1,tepig,a,0.9,0.8,0.7


In [37]:
pd.ExcelFile('Titanic Survival Model.xlsx').sheet_names

['Logistic Regression',
 'KNN',
 'XGBClassifier',
 'GBRClassifier',
 'RandomForest',
 'DecisionTree',
 'Sheet1',
 'Stacked',
 'Voting']

In [52]:
pd.read_excel('Titanic Survival Model.xlsx').sheet

ValueError: Worksheet named 'DecisionTree' not found

In [49]:
%%time
train_rf_classifier(preprocessor, RandomForestClassifier, rf_param_grid, train_df3, y_train, 'accuracy', 50,\
                         workbook, 'RandomForest', 'Titanic Survival Model.xlsx')

Max models: 50
RF SCORE: 0.6161634548992531
Num columns: 5
Current Row = 7, current column: 2, cell_value: {'randomforestclassifier__n_estimators': [100, 500, 1000], 'randomforestclassifier__min_samples_split': [10, 20, 30, 40, 50], 'randomforestclassifier__max_depth': [2, 3, 4, 5], 'randomforestclassifier__min_samples_leaf': [5, 10, 15, 20, 25, 30], 'randomforestclassifier__min_impurity_decrease': [0.002, 0.004, 0.006, 0.008]}
Current Row = 7, current column: 3, cell_value: {'n_estimators': 1000, 'min_samples_split': 30, 'min_samples_leaf': 10, 'min_impurity_decrease': 0.002, 'max_depth': 4}
Current Row = 7, current column: 4, cell_value: 0.6161634548992531
CPU times: total: 1min 47s
Wall time: 2min 21s


In [50]:
%%time
train_gbr_classifier(preprocessor, GradientBoostingClassifier, gbr_param_grid, train_df3, y_train, 'accuracy', 50,\
                         workbook, 'GBRClassifier', 'Titanic Survival Model.xlsx')

Max models: 50
Num columns: 5
Current Row = 6, current column: 2, cell_value: {'gradientboostingclassifier__n_estimators': [1000, 1200], 'gradientboostingclassifier__max_depth': [4, 5, 6], 'gradientboostingclassifier__max_leaf_nodes': [40, 30, 50], 'gradientboostingclassifier__min_samples_split': [40, 30, 50], 'gradientboostingclassifier__subsample': [0.2, 0.3, 0.4], 'gradientboostingclassifier__learning_rate': [0.03, 0.04, 0.05], 'gradientboostingclassifier__max_features': [0.01, 0.02]}
Current Row = 6, current column: 3, cell_value: {'subsample': 0.2, 'n_estimators': 1000, 'min_samples_split': 40, 'max_leaf_nodes': 50, 'max_features': 0.02, 'max_depth': 5, 'learning_rate': 0.05}
Current Row = 6, current column: 4, cell_value: 0.8249011361496453
CPU times: total: 2min 33s
Wall time: 3min 11s


In [51]:
%%time
train_knn_classifier(preprocessor, KNeighborsClassifier, knn_param_grid, train_df3, y_train, 'accuracy', 50,\
                         workbook, 'KNN', 'Titanic Survival Model.xlsx')

Max models: 50
Num columns: 5
Current Row = 6, current column: 2, cell_value: {'kneighborsclassifier__weights': ['uniform'], 'kneighborsclassifier__algorithm': ['auto'], 'kneighborsclassifier__n_neighbors': [5, 10, 15, 20, 25, 30], 'kneighborsclassifier__leaf_size': [10], 'kneighborsclassifier__p': [1]}
Current Row = 6, current column: 3, cell_value: {'weights': 'uniform', 'p': 1, 'n_neighbors': 5, 'leaf_size': 10, 'algorithm': 'auto'}
Current Row = 6, current column: 4, cell_value: 0.744121524072563
CPU times: total: 5.27 s
Wall time: 7.22 s


In [52]:
%%time
train_xgb_classifier(preprocessor, XGBClassifier, xgb_param_grid, train_df3, y_train, 'accuracy', 50,\
                         workbook, 'XGBClassifier', 'Titanic Survival Model.xlsx')

Max models: 50
Num columns: 5
Current Row = 6, current column: 2, cell_value: {'xgbclassifier__n_estimators': [110, 120], 'xgbclassifier__max_depth': [5, 6], 'xgbclassifier__max_leaves': [40, 30, 50], 'xgbclassifier__reg_alpha': [0.01, 0.02], 'xgbclassifier__reg_lambda': [0.01, 0.02], 'xgbclassifier__colsample_bytree': [0.2, 0.3, 0.4], 'xgbclassifier__learning_rate': [0.03, 0.04, 0.05], 'xgbclassifier__gamma': [0.01, 0.02]}
Current Row = 6, current column: 3, cell_value: {'reg_lambda': 0.02, 'reg_alpha': 0.01, 'n_estimators': 110, 'max_leaves': 50, 'max_depth': 6, 'learning_rate': 0.05, 'gamma': 0.02, 'colsample_bytree': 0.4}
Current Row = 6, current column: 4, cell_value: 0.8395329860021341
CPU times: total: 8min 1s
Wall time: 1min 12s


In [81]:
df = pd.read_excel('Titanic Survival Model.xlsx', sheet_name='KNN')
df

,Model Training on Source Dataset without name column,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Version,Hyperparameter Set,Best Values,Accuracy,Public Accuracy
1,1,knn_param_grid = {\n 'kneighborsclassifier_...,"{'weights': 'uniform',\n 'p': 1,\n 'n_neighb...",0.746369,NaN
2,2,knn_param_grid = {\n 'kneighborsclassifier_...,"{'weights': 'uniform', 'p': 1, 'n_neighbors': ...",0.746369,NaN
3,3,"{'kneighborsclassifier__weights': ['uniform'],...","{'weights': 'uniform', 'p': 1, 'n_neighbors': ...",0.744121524072563,NaN


In [82]:
df = pd.read_excel('Titanic Survival Model.xlsx', sheet_name='GBRClassifier')
df

,Model Training on Source Dataset without name column,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Version,Hyperparameter Set,Best Values,Accuracy,Public Accuracy
1,1,gbr_param_grid = {\n 'gradientboostingclass...,"{'subsample': 0.4,\n 'n_estimators': 1200,\n...",0.837267,NaN
2,2,gbr_param_grid = {\n 'gradientboostingclass...,"{'subsample': 0.4,\n 'n_estimators': 1200,\n '...",0.839508,NaN
3,3,{'gradientboostingclassifier__n_estimators': [...,"{'subsample': 0.4, 'n_estimators': 1000, 'min_...",0.8439771514656957,NaN


In [95]:
df = pd.read_excel('Titanic Survival Model.xlsx', sheet_name='RandomForest')
df

,Model Training on Source Dataset without name column,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Version,Hyperparameter Set,Best Values,Accuracy
1,1,rf_param_grid = {\n 'randomforestclassifier...,"{'n_estimators': 1000,\n 'min_samples_split':...","0.6161634548992531,"
2,2,rf_param_grid = {\n 'randomforestclassifier...,"{'n_estimators': 100,\n 'min_samples_split': ...",0.616163
3,3,rf_param_grid = {\n 'randomforestclassifier...,"{'n_estimators': 100, 'min_samples_split': 50,...",0.616163
4,4,"{'randomforestclassifier__n_estimators': [100,...","{'n_estimators': 100, 'min_samples_split': 30,...",NaN


### Feature Engineering

In [33]:
def is_married(x):
    return 1 if 'mrs.' in x['Name'].lower() and x['Sex'] == 'female' else 0

def get_title(x):
    return x.split(', ')[1].split(' ')[0]

def get_age_group(age):
    if age < 30:
        return 0
    else:
        return 1
    
def convert_title(title):
    if title not in ['Mr.', 'Miss.', 'Mrs.', 'Master']:
        return 'other'
    else:
        return title

In [34]:
train_df4 = pd.read_csv('train.csv')
test_df4 = pd.read_csv('test.csv')

In [35]:
train_df4['is_married'] = train_df4.apply(lambda x: is_married(x), axis='columns')
train_df4['title'] = train_df4['Name'].apply(lambda x: get_title(x))
train_df4['title_converted'] = train_df4['title'].apply(lambda x: convert_title(x))
train_df4['sex_converted'] = train_df4['Sex'].apply(lambda x: 1 if x == 'male' else 0)
train_df4['age_group'] = train_df4['Age'].apply(lambda x: get_age_group(x))
train_df4['family_size'] = train_df4['Parch'] + train_df4['SibSp']
train_df4['solo_traveler'] = train_df4['family_size'].apply(lambda x: 1 if x == 0 else 0)

In [36]:
test_df4['is_married'] = test_df4.apply(lambda x: is_married(x), axis='columns')
test_df4['title'] = test_df4['Name'].apply(lambda x: get_title(x))
test_df4['title_converted'] = test_df4['title'].apply(lambda x: convert_title(x))
test_df4['sex_converted'] = test_df4['Sex'].apply(lambda x: 1 if x == 'male' else 0)
test_df4['age_group'] = test_df4['Age'].apply(lambda x: get_age_group(x))
test_df4['family_size'] = test_df4['Parch'] + test_df4['SibSp']
test_df4['solo_traveler'] = test_df4['family_size'].apply(lambda x: 1 if x == 0 else 0)

In [37]:
nec_variables = ['Pclass', 'Embarked', 'Fare','is_married', 'title_converted', 'sex_converted', 'age_group', 'Age', 'family_size', 'solo_traveler', 'Parch', 'SibSp']

In [38]:
train_df5 = train_df4[nec_variables]
test_df5 = test_df4[nec_variables]
y_train = train_df[label]

In [39]:
train_df5['is_married'].unique()

array([0, 1], dtype=int64)

In [40]:
train_df5.dtypes

Pclass               int64
Embarked            object
Fare               float64
is_married           int64
title_converted     object
sex_converted        int64
age_group            int64
Age                float64
family_size          int64
solo_traveler        int64
Parch                int64
SibSp                int64
dtype: object

In [56]:
preprocessor2 = get_preprocessor(train_df5, id_column, label)

Continuous Variables: ['Fare', 'Age']
Discrete Numerical Variables: ['is_married', 'solo_traveler', 'sex_converted', 'family_size', 'SibSp', 'age_group', 'Pclass', 'Parch']
Categorical Variables: ['Embarked', 'title_converted']


In [42]:
#len(['Fare', 'Age'] + ['is_married', 'solo_traveler', 'sex_converted', 'family_size', 'SibSp', 'age_group', 'Pclass', 'Parch'] + ['Embarked', 'title_converted'])

12

In [57]:
%%time
dt_final_pipe = train_dt_classifier(preprocessor2, DecisionTreeClassifier, dt_param_grid, train_df5, y_train, 'accuracy', 50,\
                         workbook, 'DecisionTree', 'Titanic Survival Model.xlsx')

Max models: 50
DT score: 0.7150398593936351
Num columns: 5
Current Row = 9, current column: 2, cell_value: {'decisiontreeclassifier__max_depth': [2, 3], 'decisiontreeclassifier__min_samples_split': [10, 20, 30], 'decisiontreeclassifier__min_samples_leaf': [10, 20, 30], 'decisiontreeclassifier__max_features': ['sqrt'], 'decisiontreeclassifier__min_impurity_decrease': [0.01, 0.02], 'decisiontreeclassifier__ccp_alpha': [0.1, 0.2]}
Current Row = 9, current column: 3, cell_value: {'min_samples_split': 20, 'min_samples_leaf': 20, 'min_impurity_decrease': 0.02, 'max_features': 'sqrt', 'max_depth': 2, 'ccp_alpha': 0.1}
Current Row = 9, current column: 4, cell_value: 0.7150398593936351
CPU times: total: 2.58 s
Wall time: 3.61 s


In [63]:
%%time
rf_final_pipe = train_rf_classifier(preprocessor2, RandomForestClassifier, rf_param_grid, train_df5, y_train, 'accuracy', 50,\
                         workbook, 'RandomForest', 'Titanic Survival Model.xlsx')

Max models: 50
RF SCORE: 0.8159186491745652
Num columns: 5
Current Row = 9, current column: 2, cell_value: {'randomforestclassifier__n_estimators': [100, 500, 1000], 'randomforestclassifier__min_samples_split': [10, 20, 30, 40, 50], 'randomforestclassifier__max_depth': [2, 3, 4, 5], 'randomforestclassifier__min_samples_leaf': [5, 10, 15, 20, 25, 30], 'randomforestclassifier__min_impurity_decrease': [0.002, 0.004, 0.006, 0.008]}
Current Row = 9, current column: 3, cell_value: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 10, 'min_impurity_decrease': 0.004, 'max_depth': 4}
Current Row = 9, current column: 4, cell_value: 0.8159186491745652
CPU times: total: 1min 23s
Wall time: 2min 6s


In [64]:
%%time
knn_final_pipe = train_knn_classifier(preprocessor2, KNeighborsClassifier, knn_param_grid, train_df5, y_train, 'accuracy', 50,\
                         workbook, 'KNN', 'Titanic Survival Model.xlsx')

Max models: 50
Num columns: 5
Current Row = 10, current column: 2, cell_value: {'kneighborsclassifier__weights': ['uniform'], 'kneighborsclassifier__algorithm': ['auto'], 'kneighborsclassifier__n_neighbors': [5, 10, 15, 20, 25, 30], 'kneighborsclassifier__leaf_size': [10], 'kneighborsclassifier__p': [1]}
Current Row = 10, current column: 3, cell_value: {'weights': 'uniform', 'p': 1, 'n_neighbors': 5, 'leaf_size': 10, 'algorithm': 'auto'}
Current Row = 10, current column: 4, cell_value: 0.7508819283158622
CPU times: total: 438 ms
Wall time: 631 ms


In [65]:
%%time
xgb_final_pipe = train_xgb_classifier(preprocessor2, XGBClassifier, xgb_param_grid, train_df5, y_train, 'accuracy', 50,\
                         workbook, 'XGBClassifier', 'Titanic Survival Model.xlsx')

Max models: 50
Num columns: 5
Current Row = 8, current column: 2, cell_value: {'xgbclassifier__n_estimators': [110, 120], 'xgbclassifier__max_depth': [5, 6], 'xgbclassifier__max_leaves': [40, 30, 50], 'xgbclassifier__reg_alpha': [0.01, 0.02], 'xgbclassifier__reg_lambda': [0.01, 0.02], 'xgbclassifier__colsample_bytree': [0.2, 0.3, 0.4], 'xgbclassifier__learning_rate': [0.03, 0.04, 0.05], 'xgbclassifier__gamma': [0.01, 0.02]}
Current Row = 8, current column: 3, cell_value: {'reg_lambda': 0.01, 'reg_alpha': 0.01, 'n_estimators': 120, 'max_leaves': 30, 'max_depth': 6, 'learning_rate': 0.04, 'gamma': 0.02, 'colsample_bytree': 0.3}
Current Row = 8, current column: 4, cell_value: 0.8395204318624068
CPU times: total: 1min 47s
Wall time: 16.7 s


In [66]:
%%time
gbr_final_pipe = train_gbr_classifier(preprocessor2, GradientBoostingClassifier, gbr_param_grid, train_df5, y_train, 'accuracy', 50,\
                         workbook, 'GBRClassifier', 'Titanic Survival Model.xlsx')

Max models: 50
Num columns: 5
Current Row = 8, current column: 2, cell_value: {'gradientboostingclassifier__n_estimators': [1000, 1200], 'gradientboostingclassifier__max_depth': [4, 5, 6], 'gradientboostingclassifier__max_leaf_nodes': [40, 30, 50], 'gradientboostingclassifier__min_samples_split': [40, 30, 50], 'gradientboostingclassifier__subsample': [0.2, 0.3, 0.4], 'gradientboostingclassifier__learning_rate': [0.03, 0.04, 0.05], 'gradientboostingclassifier__max_features': [0.01, 0.02]}
Current Row = 8, current column: 3, cell_value: {'subsample': 0.3, 'n_estimators': 1000, 'min_samples_split': 40, 'max_leaf_nodes': 40, 'max_features': 0.02, 'max_depth': 5, 'learning_rate': 0.04}
Current Row = 8, current column: 4, cell_value: 0.8237963718536188
CPU times: total: 2min 8s
Wall time: 3min 1s


In [67]:
lr_final_pipe = make_pipeline(preprocessor2, LogisticRegression())

In [69]:
 estimators = [
        ('LogisticRegression', lr_final_pipe),
         ('KNN', knn_final_pipe),
        ('RandomForest', rf_final_pipe),
        ('GradientBoostingClassifier', gbr_final_pipe),
        ('XGBClassifier', xgb_final_pipe)
]

In [80]:
rf_final_estimator = RandomForestClassifier()
xgb_final_estimator = XGBClassifier()

In [71]:
rf_stack, rf_stack_score = train_stacking_classifier(train_df5, y_train, estimators, rf_final_estimator, 'accuracy')
#predict(train_df5, y_train, test_df5, rf_stack, id_column,test_df[id_column], pred_col, label, 'rf_stack_feat_1.csv')

Score: 0.8136902893729208


In [81]:
xgb_stack, xgb_stack_score = train_stacking_classifier(train_df5, y_train, estimators, xgb_final_estimator, 'accuracy')
xgb_stack_score

Score: 0.7789529847467203


0.7789529847467203

In [82]:
predict(train_df5, y_train, test_df5, xgb_stack, id_column, test_df[id_column], pred_col, label, 'xgb_stack_feat_1.csv')

### Voting

In [91]:
vote_rf = VotingClassifier(estimators=[('LogisticRegression', lr_final_pipe),\
                                         ('KNN', knn_final_pipe),
                                         ('RF', rf_final_pipe),
                                        ('GBR', gbr_final_pipe),
                                        ('XGB', xgb_final_pipe)], voting='hard')

score = cross_val_score(vote_rf, train_df5, y_train, cv=5, scoring='accuracy').mean()

print(f'Voting Classifier with RF score: {score}')

Voting Classifier with RF score: 0.8316489862532169


In [92]:
predict(train_df5, y_train, test_df5, vote_rf, id_column, test_df[id_column], pred_col, label, 'vote_rf_feat_1.csv')

In [83]:
 estimators_no_rf = [
        ('LogisticRegression', lr_final_pipe),
         ('KNN', knn_final_pipe),
        ('GradientBoostingClassifier', gbr_final_pipe),
        ('XGBClassifier', xgb_final_pipe)
]

In [86]:
rf_stack_no_rf, rf_stack_score = train_stacking_classifier(train_df5, y_train, estimators_no_rf, rf_final_estimator, 'accuracy')

Score: 0.8069675475488042


In [87]:
xgb_stack_no_rf, xgb_stack_score = train_stacking_classifier(train_df5, y_train, estimators_no_rf, xgb_final_estimator, 'accuracy')


Score: 0.7710878162073944


In [88]:
predict(train_df5, y_train, test_df5, rf_stack_no_rf, id_column,test_df[id_column], pred_col, label, 'rf_stack_no_rf_feat_1.csv')

In [89]:
predict(train_df5, y_train, test_df5, xgb_stack_no_rf, id_column,test_df[id_column], pred_col, label, 'xgb_stack_no_rf_feat_1.csv')

In [93]:
vote_no_rf = VotingClassifier(estimators=[('LogisticRegression', lr_final_pipe),\
                                         ('KNN', knn_final_pipe),
                                        ('GBR', gbr_final_pipe),
                                        ('XGB', xgb_final_pipe)], voting='hard')

score = cross_val_score(vote_no_rf, train_df5, y_train, cv=5, scoring='accuracy').mean()

print(f'Voting Classifier with no RF score: {score}')

Voting Classifier with no RF score: 0.8294080723118448


In [94]:
predict(train_df5, y_train, test_df5, vote_no_rf, id_column, test_df[id_column], pred_col, label, 'vote_no_rf_feat_1.csv')

In [97]:
nec_variables

['Pclass',
 'Embarked',
 'Fare',
 'is_married',
 'title_converted',
 'sex_converted',
 'age_group',
 'Age',
 'family_size',
 'solo_traveler',
 'Parch',
 'SibSp']

In [96]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [100]:
train_df6 = train_df4[nec_variables + ['Name', 'Ticket', 'Cabin']]
test_df6 = test_df4[nec_variables + ['Name', 'Ticket', 'Cabin']]
y_train = train_df[label]

In [102]:
preprocessor3 = get_preprocessor(train_df6, id_column, label)

Continuous Variables: ['Fare', 'Age']
Discrete Numerical Variables: ['is_married', 'solo_traveler', 'sex_converted', 'family_size', 'SibSp', 'age_group', 'Pclass', 'Parch']
Categorical Variables: ['Embarked', 'title_converted', 'Ticket', 'Name', 'Cabin']


In [38]:
x = ['Embarked', 'title_converted', 'Ticket', 'Name', 'Cabin']
',\n '.join(x)

'Embarked,\n title_converted,\n Ticket,\n Name,\n Cabin'

#### Using all variables

In [103]:
%%time
dt_final_pipe = train_dt_classifier(preprocessor3, DecisionTreeClassifier, dt_param_grid, train_df6, y_train, 'accuracy', 50,\
                         workbook, 'DecisionTree', 'Titanic Survival Model.xlsx')

rf_final_pipe = train_rf_classifier(preprocessor3, RandomForestClassifier, rf_param_grid, train_df6, y_train, 'accuracy', 50,\
                         workbook, 'RandomForest', 'Titanic Survival Model.xlsx')

knn_final_pipe = train_knn_classifier(preprocessor3, KNeighborsClassifier, knn_param_grid, train_df6, y_train, 'accuracy', 50,\
                         workbook, 'KNN', 'Titanic Survival Model.xlsx')

xgb_final_pipe = train_xgb_classifier(preprocessor3, XGBClassifier, xgb_param_grid, train_df6, y_train, 'accuracy', 50,\
                         workbook, 'XGBClassifier', 'Titanic Survival Model.xlsx')

gbr_final_pipe = train_gbr_classifier(preprocessor3, GradientBoostingClassifier, gbr_param_grid, train_df6, y_train, 'accuracy', 50,\
                         workbook, 'GBRClassifier', 'Titanic Survival Model.xlsx')

Max models: 50
DT score: 0.6161634548992531
Num columns: 5
Current Row = 10, current column: 2, cell_value: {'decisiontreeclassifier__max_depth': [2, 3], 'decisiontreeclassifier__min_samples_split': [10, 20, 30], 'decisiontreeclassifier__min_samples_leaf': [10, 20, 30], 'decisiontreeclassifier__max_features': ['sqrt'], 'decisiontreeclassifier__min_impurity_decrease': [0.01, 0.02], 'decisiontreeclassifier__ccp_alpha': [0.1, 0.2]}
Current Row = 10, current column: 3, cell_value: {'min_samples_split': 20, 'min_samples_leaf': 30, 'min_impurity_decrease': 0.02, 'max_features': 'sqrt', 'max_depth': 2, 'ccp_alpha': 0.1}
Current Row = 10, current column: 4, cell_value: 0.6161634548992531
Max models: 50
RF SCORE: 0.6161634548992531
Num columns: 5
Current Row = 10, current column: 2, cell_value: {'randomforestclassifier__n_estimators': [100, 500, 1000], 'randomforestclassifier__min_samples_split': [10, 20, 30, 40, 50], 'randomforestclassifier__max_depth': [2, 3, 4, 5], 'randomforestclassifier__m

In [104]:
lr_final_pipe = make_pipeline(preprocessor3, LogisticRegression())

In [105]:
 estimators = [
        ('LogisticRegression', lr_final_pipe),
         ('KNN', knn_final_pipe),
        ('RandomForest', rf_final_pipe),
        ('GradientBoostingClassifier', gbr_final_pipe),
        ('XGBClassifier', xgb_final_pipe)
]

In [106]:
rf_final_estimator = RandomForestClassifier()
xgb_final_estimator = XGBClassifier()

In [107]:
rf_stack, rf_stack_score = train_stacking_classifier(train_df6, y_train, estimators, rf_final_estimator, 'accuracy')

Score: 0.8114870378507314


In [108]:
predict(train_df6, y_train, test_df6, rf_stack, id_column, test_df[id_column], pred_col, label, 'rf_stack_feat_2.csv')

In [110]:
vote_rf = VotingClassifier(estimators=[('LogisticRegression', lr_final_pipe),\
                                         ('KNN', knn_final_pipe),
                                           ('RF', rf_final_pipe),
                                        ('GBR', gbr_final_pipe),
                                        ('XGB', xgb_final_pipe)], voting='hard')

score = cross_val_score(vote_rf, train_df6, y_train, cv=5, scoring='accuracy').mean()

print(f'Voting Classifier with RF score: {score}')

Voting Classifier with RF score: 0.8394827694432239


In [111]:
predict(train_df6, y_train, test_df6, vote_rf, id_column, test_df[id_column], pred_col, label, 'vote_feat_2.csv')

In [114]:
train_df['Cabin'].isna().sum()

687

In [116]:
train_df['Ticket'].isna().sum()

0

In [115]:
train_df.shape

(891, 12)

In [117]:
train_df['Ticket'].unique()

array(['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450',
       '330877', '17463', '349909', '347742', '237736', 'PP 9549',
       '113783', 'A/5. 2151', '347082', '350406', '248706', '382652',
       '244373', '345763', '2649', '239865', '248698', '330923', '113788',
       '347077', '2631', '19950', '330959', '349216', 'PC 17601',
       'PC 17569', '335677', 'C.A. 24579', 'PC 17604', '113789', '2677',
       'A./5. 2152', '345764', '2651', '7546', '11668', '349253',
       'SC/Paris 2123', '330958', 'S.C./A.4. 23567', '370371', '14311',
       '2662', '349237', '3101295', 'A/4. 39886', 'PC 17572', '2926',
       '113509', '19947', 'C.A. 31026', '2697', 'C.A. 34651', 'CA 2144',
       '2669', '113572', '36973', '347088', 'PC 17605', '2661',
       'C.A. 29395', 'S.P. 3464', '3101281', '315151', 'C.A. 33111',
       'S.O.C. 14879', '2680', '1601', '348123', '349208', '374746',
       '248738', '364516', '345767', '345779', '330932', '113059',
       'SO/C 14885', '31012